<h1 style="text-align: center; font-size: 50px;"> Agentic RAG </h1>

This notebook showcases a **Hugging Face** model integrated with a **retriever tool**, enabling it to fetch and use relevant context dynamically when answering questions about **Z by HP AI Studio**.  

The solution is primarily built using the **LangChain** and **SmolAgents** libraries, creating an agent capable of context-aware retrieval and response generation.

# Notebook Overview

- Imports
- Configurations
- Verify Assets
- Load PDF File
- Split Text
- Embed Text
- Define Retriever Tool
- Create Agent
- Run Query

# Imports

In [ ]:
%pip install -r ../requirements.txt --quiet

In [ ]:
import os
from typing import List
from tqdm import tqdm
from PyPDF2 import PdfReader

from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.vectorstores import VectorStore
from smolagents import Tool, HfApiModel, ToolCallingAgent

# Configurations

In [ ]:
# Suppress Python warnings
warnings.filterwarnings("ignore")

In [ ]:
# Create logger
logger = logging.getLogger("notebook_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %H:%M:%S")  

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [ ]:
PDF_PATH = "../data/AIStudioDoc.pdf"
TOKENIZER_NAME = "thenlper/gte-small"
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
HF_MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [ ]:
# Replace with your actual Hugging Face API key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [ ]:
logger.info('Notebook execution started.')

# Load PDF File

In [ ]:
reader = PdfReader(PDF_PATH)

print("Reading and extracting PDF content...")
source_docs = []
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        source_docs.append(Document(page_content=text, metadata={"source": f"page_{i + 1}"}))

# Split Text

In [ ]:
print("Splitting documents...")
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained(TOKENIZER_NAME),
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[new_doc.page_content] = True
            docs_processed.append(new_doc)


# Embed Text

In [ ]:
print("Embedding documents...")
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
)

# Define Retriever Tool

In [ ]:
class RetrieverTool(Tool):
    name = "retriever"
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"
        docs = self.vectordb.similarity_search(query, k=7)
        return "\nRetrieved documents:\n" + "".join(
            [f"===== Document {i} =====\n" + doc.page_content for i, doc in enumerate(docs)]
                )

# Create Agent

In [ ]:
model = HfApiModel(HF_MODEL_NAME)
retriever_tool = RetrieverTool(vectordb)
agent = ToolCallingAgent(tools=[retriever_tool], model=model)

# Run Query

In [ ]:
question = "What is the Z by HP AI Studio?"

enhanced_question = f"""Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should not be questions but affirmative form sentences: e.g. rather than "What is the Z by HP AI Studio?", query should be "The Z by HP AI Studio is a platform...".

Question:
{question}"""

answer = agent.run(enhanced_question)
print(answer)


In [ ]:
logger.info('Notebook execution completed.')

Built with ❤️ using [**Z by HP AI Studio**](https://zdocs.datascience.hp.com/docs/aistudio/overview).